In [3]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable, Tuple

Метод золотого сечения

In [ ]:
def golden_section_search(
    f: Callable[[float], float], a: float, b: float, epsilon: float = 1e-6, max_iter: int = 100
) -> Tuple[float, float]:

    K = (np.sqrt(5) - 1) / 2
    L0 = b - a
    L1 = K * L0
    x = b - L1
    y = a + L1
    
    fx = f(x)
    fy = f(y)
    
    for k in range(1, max_iter + 1):
        L_next = K * (b - a)
        
        if fx < fy:
            b = y
            y = x
            fy = fx
            x = b - L_next
            fx = f(x)
        else:
            a = x
            x = y
            fx = fy
            y = a + L_next
            fy = f(y)
        
        if L_next < epsilon:
            if fx < fy:
                x_min = x
                f_min = fx
            else:
                x_min = y
                f_min = fy
            return x_min, f_min
    
    if fx < fy:
        return x, fx
    else:
        return y, fy


Найденный минимум: x = -1.090170, f(x) = 0.008131
Ожидаемый минимум: x = -1.000000, f(x) = 0.000000


Метод парабол

Метод Брента

In [ ]:
def f1(x: float):
    return -5*x**5 + 4*x**4 - 12*x**3 + 11*x**2 - 2*x + 1

def f2(x: float):
    return -(np.log(x-2))**2 + (np.log(10-x))**2 - x**0.2

def f3(x: float):
    return -3*x * np.sin(0.75*x) + np.exp(-2*x)

def f4(x: float):
    return np.exp(3*x) + 5*np.exp(-2*x)

def f5(x: float):
    return 0.2*x * np.log(x) + (x-2.3)**2